In [ ]:
import pandas as pd

In [108]:
poleval_df = pd.read_csv("poleval2019_task2_training_190221/index.tsv", sep="\t", header=None)
poleval_df.columns = ["phrase_id", "doc_id", "text", "lemma"]
poleval_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,40025,99883,Toronto,Toronto
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
4,327365,99883,kanadyjskim,kanadyjski


In [109]:
print(f"Shape of the original Poleval 2019 dataset: {poleval_df.shape}")

Shape of the original Poleval 2019 dataset: (22177, 4)


In [110]:
poleval_df = poleval_df[poleval_df["text"] != poleval_df["lemma"]]

In [111]:
print(f"Shape of the Poleval 2019 dataset (only inflected expressions): {poleval_df.shape}")

Shape of the Poleval 2019 dataset (only inflected expressions): (10720, 4)


In [112]:
mwe_df = poleval_df[poleval_df["text"].str.split().str.len() > 1]
mwe_df.head()

,phrase_id,doc_id,text,lemma
5,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
18,343879,100499,samolotu PZL.23 „Karaś”,samolot PZL.23 „Karaś”
19,31843,100499,Ministerstwa Komunikacji,Ministerstwo Komunikacji
22,343880,100499,silnikiem Pratt-Whitney Wasp,silnik Pratt-Whitney Wasp
28,31839,100499,Stanisława Praussa,Stanisław Prauss


In [113]:
print(f"Shape of the Poleval 2019 dataset (only inflected multi word expressions): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (only inflected multi word expressions): (4783, 4)


In [114]:
mwe_df[['text', 'lemma']].sample(10)

,text,lemma
3874,województwie małopolskim,województwo małopolskie
14546,Wielkiej Orkiestry,Wielka Orkiestra
2020,buddyzmie tybetańskim,buddyz tybetański
21843,Państwowej Straży Pożarnej,Państwowa Straż Pożarna
15326,XXII Otwartych Halowych Mistrzostwach Łodzi i Regionu Łódzkiego TKKF Golden Cup 2007,XXII Otwarte Halowe Mistrzostwa Łodzi i Regionu Łódzkiego TKKF Golden Cup 2007
12447,Jaskini Mroźnej,Jaskinia Mroźna
6838,III Światowym Festiwalu Młodzieży i Studentów w Budapeszcie,III Światowy Festiwal Młodzieży i Studentów w Budapeszcie
8070,akcji „Burza”,akcja „Burza”
8105,drugiej wojny światowej,druga wojna światowa
10335,ochronie środowiska,ochrona środowiska


In [115]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        1
dtype: int64

In [116]:
mwe_df[mwe_df["lemma"].isna()]

,phrase_id,doc_id,text,lemma
3332,307628,101286,San Diego Rockets,NaN


In [117]:
# I can drop it because it's not a polish MWE
mwe_df = mwe_df.dropna()

In [118]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        0
dtype: int64

In [119]:
len(mwe_df["lemma"].unique())

3649

In [120]:
mwe_df[mwe_df["text"] == "Dolce & Gabbana"]

,phrase_id,doc_id,text,lemma
401,99725,100524,Dolce & Gabbana,Dolce &amp; Gabbana


In [121]:
mwe_df[mwe_df["text"] == "Lublin R-VIII a"]

,phrase_id,doc_id,text,lemma
1830,65875,101200,Lublin R-VIII a,Lublin R-VIII


In [122]:
mwe_df[mwe_df["text"] == "BBC Radio 1's Live Lounge"]

,phrase_id,doc_id,text,lemma
5561,426217,101434,BBC Radio 1's Live Lounge,BBC Radio 1


In [123]:
mwe_df[mwe_df["text"] == "Jammin' CRT.5"]

,phrase_id,doc_id,text,lemma
12269,468926,102125,Jammin' CRT.5,Jammin


In [124]:
mwe_df[mwe_df["text"] == "Traxxas Rustler XL-1 '09"]


,phrase_id,doc_id,text,lemma
15591,469441,102506,Traxxas Rustler XL-1 '09,Traxxas Rustler XL-1


In [125]:
mwe_df[mwe_df["text"] == "Żerań F S O"]


,phrase_id,doc_id,text,lemma
16120,350246,102673,Żerań F S O,Żerań FSO


In [126]:
mwe_df.loc[mwe_df["text"] == "Żerań F S O", "text"] = "Żerań FSO"

In [127]:
mwe_df = mwe_df[mwe_df["text"] != "S K M"]
mwe_df = mwe_df[mwe_df["text"] != "R K S"]

In [128]:
mwe_df[mwe_df["text"] == "átlátszó ("]

,phrase_id,doc_id,text,lemma
18833,481079,103921,átlátszó (,átlátszó


In [129]:
mwe_df[mwe_df["text"] == "Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”"]

,phrase_id,doc_id,text,lemma
12540,79818,102157,Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”,Społeczna Rada Konsultacyjna ds. aktualizacji„Strategii Rozwoju Bydgoszczy do 2015 roku


In [130]:
mwe_df.loc[mwe_df["text"] == 'Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”', "lemma"] = 'Społeczna Rada Konsultacyjna ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku"'

In [131]:
mwe_df[mwe_df["text"].str.contains("Fundacji Ośrodka")]

,phrase_id,doc_id,text,lemma
15959,71729,102602,"Fundacji Ośrodka "" Karta ""","Fundacja Ośrodka ""Karta """


In [132]:
mwe_df.loc[mwe_df["text"] == 'Fundacji Ośrodka " Karta "', ["text", "lemma"]] = ['Fundacji Ośrodka "Karta"', 'Fundacja Ośrodka "Karta"']

In [133]:
mwe_df[mwe_df["text"].str.contains("FILLER")] # error during creating dataset

,phrase_id,doc_id,text,lemma
16262,474289,102694,Dworca FILLER Centralnego,Dworzec Centralny
16274,350524,102696,Dworzec FILLER Śródmieście,Dworzec Śródmieście


In [134]:
mwe_df.loc[mwe_df["text"] == 'Dworca FILLER Centralnego', "text"] = 'Dworca Centralnego'
mwe_df.loc[mwe_df["text"] == 'Dworzec FILLER Śródmieście', "text"] = 'Dworzec Śródmieście'

In [135]:
mwe_df[mwe_df["text"].str.contains("Herburt")]

,phrase_id,doc_id,text,lemma
3021,50764,101263,Jan Herburt (Arłamowski,Jan Herburt (Arłamowski)


In [136]:
mwe_df.drop(mwe_df[mwe_df["text"].isin([
    "Dolce & Gabbana", # not a polish MWE
    "Lublin R-VIII a", # ???
    "BBC Radio 1's Live Lounge", # not a polish MWE
    "Jammin' CRT.5", # not a polish MWE
    "Traxxas Rustler XL-1 '09", # not a polish MWE
    "átlátszó (" # not a polish MWE
])].index, inplace=True)

In [137]:
from morfeusz2 import Morfeusz
morfeusz = Morfeusz()  

def get_case_and_number(phrase, lemma):
    """
    Function to get case and number of a word using Morfeusz2.
    :param phrase: str, phrase to be analyzed
    :param lemma: str, lemma of the phrase
    :return: str, number and case of the word
    """
    phrase = phrase.replace(" - ", "-")
    phrase = phrase.replace("___", "")

    
    words = phrase.strip().split()
    lemmas = lemma.strip().split() 


    # possible numbers and cases for the word
    number_case = {}
    part_of_speech = {}

    # returned values
    number = None
    case = None
    relation = None
    head = None

    for i, word in enumerate(words):
        number_case_i = set()
        analyses = morfeusz.analyse(word)

        for _, _, interpretations in analyses:
            tag = interpretations[2].split(":")
            if len(tag) > 2:
                number_i = tag[1]
                case_i = tag[2].split('.')

                # Morpheu can return more than 1 possibilty of case after the dot so I make  alist of it
                for c in case_i:
                    number_case_i.add((number_i, c))

                part_of_speech[word] = tag[0]

        number_case[i] = number_case_i

    # check if it is possible to find common number and case for the words in the mwe -> ZWIĄZEK ZGODY
    common_cases = set.intersection(*number_case.values())
    if len(common_cases)>=1:
        # if there is only one common pair(number, case), then it is an agreement relation and the number and case of the expression can be assigned to the whole expression
        number, case = list(common_cases)[0]
        relation = "agreement"

        # noun -> HEAD
        for i, (word, tag) in enumerate(part_of_speech.items()):
            if tag == 'subst':
                head = lemmas[i]
                break

    else:
        if [w_i for w_i, l_i in zip(words, lemmas) if w_i == l_i]:
            # government relation    
            for i, word in enumerate(words):
                #print(f"word: {word}, {i}, {words}, {lemmas}")
                if word == lemmas[i]:
                    continue
                else: 
                    if number_case[i]:
                        head = lemmas[i]
                        number_case_head_list = list(number_case[i])
                        #print(f"head: {head}, number_case_head: {number_case_head}")
                        number = [x[0] for x in number_case_head_list]
                        case = [x[1] for x in number_case_head_list]
                        relation = "government"
                        # print(f"number: {number}, case: {case}")
        
    return number, case, relation, head
            

In [138]:
mwe_df[["number", "case", "relation", "head"]] = mwe_df.apply(lambda row: pd.Series(get_case_and_number(row["text"], row["lemma"])), axis=1)

In [139]:
mwe_df.sample(10)

,phrase_id,doc_id,text,lemma,number,case,relation,head
7641,466794,101594,Radę Ministrów,Rada Ministrów,[sg],[acc],government,Rada
1984,53564,101213,św. Jerzego,św. Jerzy,"[sg, sg]","[acc, gen]",government,Jerzy
7794,63705,101614,Fejdolosa Koryntianina,Fejdolos Koryntianin,None,None,None,None
6627,428051,101522,rynku pracy,rynek pracy,sg,loc,agreement,rynek
14841,469119,102428,tygodniku “Polityka”,tygodnik “Polityka”,"[sg, sg]","[voc, loc]",government,tygodnik
6527,427974,101509,społecznej teorii uczenia się,społeczna teoria uczenia się,"[sg, pl, sg, sg]","[dat, gen, loc, gen]",government,teoria
18160,468298,103837,W. Mullera,W. Muller,"[sg, sg]","[acc, gen]",government,Muller
1429,352293,101173,Mistrzostwach Świata w Narciarstwie Klasycznym,Mistrzostwa Świata w Narciarstwie Klasycznym,[pl],[loc],government,Mistrzostwa
12637,466319,102161,województwa mazowieckiego,województwo mazowieckie,sg,gen,agreement,województwo
16996,467997,103609,Kodeksu handlowego z 1934 r,Kodeks handlowy z 1934 r,"[sg, sg]","[acc, gen]",government,handlowy


In [140]:
mwe_df.isna().sum()

phrase_id      0
doc_id         0
text           0
lemma          0
number       382
case         382
relation     382
head         463
dtype: int64

In [141]:
mwe_df.shape

(4774, 8)

In [142]:
mwe_df[mwe_df["case"].isna()]

,phrase_id,doc_id,text,lemma,number,case,relation,head
58,31877,100502,Carla Gotharda Langhansa,Carl Gothard Langhans,None,None,None,None
323,32169,100521,Wojen mandaloriańskich,wojny mandaloriańskie,None,None,None,None
423,344047,100525,Johanna Mühlegga,Johann Mühlegga,None,None,None,None
452,345762,100527,archipelagu Seszeli,archipelag Seszele,None,None,None,None
580,32501,100538,Janiną Milejkowską,Janina Milejkowska,None,None,None,None
...,...,...,...,...,...,...,...,...
21462,326487,107334,Harrim Ollim,Harri Olli,None,None,None,None
21466,326488,107334,Simonem Ammannem,Simon Ammann,None,None,None,None
21574,326940,107340,Nuriego al-Malikiego,Nuri al-Maliki,None,None,None,None
21599,490565,107342,prezydenta Obamy,prezydent Obama,None,None,None,None


In [144]:
morfeusz.analyse("prezydenta Obamy")

[(0,
  1,
  ('prezydenta',
   'prezydent:Sm1',
   'subst:sg:gen.acc:m1',
   ['nazwa_pospolita'],
   [])),
 (1, 2, ('Obamy', 'Obamy', 'ign', [], []))]

Problematic expressions:

- "Międzynarodową Sanghę" (there's no word "Sangha" in Morfeusz)

- 